In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model_path = "/data/yanghq/models/meta-llama/Llama-3.2-3B-Instruct"
model_path = "/data/yanghq/models/custom/svd-30"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)
model = model.to(device)
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(model_path)

input_text = "Hello! How are you?"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(**inputs, max_length=25)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Loading checkpoint shards: 100%|██████████| 2/2 [01:31<00:00, 45.94s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Hello! How are you? and are not. and are not. and are not. and are not. and are


In [1]:
from vllm import LLM, SamplingParams

model_path = "/data/yanghq/models/meta-llama/Llama-3.2-3B-Instruct"

prompts = [
    "Hello, my name is",
    "The president of the United States is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95,max_tokens=40)
llm = LLM(
    model=model_path,
    trust_remote_code=True,
    dtype='bfloat16',
    gpu_memory_utilization=0.7,
    max_model_len=2048,
)
outputs = llm.generate(prompts, sampling_params)
print("\nGenerated Outputs:\n" + "-" * 6)
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt:    {prompt!r}")
    print(f"Output:    {generated_text!r}")
    print("-" * 6)


/data/yanghq/utils/anaconda3/envs/svd/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-13 13:39:11 [__init__.py:244] Automatically detected platform cuda.


2025-06-13 13:39:13,356	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 06-13 13:39:22 [config.py:823] This model supports multiple tasks: {'embed', 'score', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 06-13 13:39:22 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-13 13:39:24 [core.py:455] Waiting for init message from front-end.
INFO 06-13 13:39:24 [core.py:70] Initializing a V1 LLM engine (v0.9.2.dev1+gae5efe7e6) with config: model='/data/yanghq/models/meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='/data/yanghq/models/meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='a

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  4.11it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.62it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.78it/s]



INFO 06-13 13:39:27 [default_loader.py:272] Loading weights took 1.29 seconds
INFO 06-13 13:39:27 [gpu_model_runner.py:1625] Model loading took 6.0160 GiB and 1.587052 seconds
INFO 06-13 13:39:33 [backends.py:462] Using cache directory: /data/yanghq/.cache/vllm/torch_compile_cache/a2bf5b80fb/rank_0_0 for vLLM's torch.compile
INFO 06-13 13:39:33 [backends.py:472] Dynamo bytecode transform time: 5.96 s
INFO 06-13 13:39:38 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 4.406 s
INFO 06-13 13:39:39 [monitor.py:34] torch.compile takes 5.96 s in total
INFO 06-13 13:39:40 [gpu_worker.py:227] Available KV cache memory: 9.34 GiB
INFO 06-13 13:39:40 [kv_cache_utils.py:715] GPU KV cache size: 87,408 tokens
INFO 06-13 13:39:40 [kv_cache_utils.py:719] Maximum concurrency for 2,048 tokens per request: 42.68x
INFO 06-13 13:39:56 [gpu_model_runner.py:2049] Graph capturing finished in 16 secs, took 1.41 GiB
INFO 06-13 13:39:56 [core.py:171] init engine (profile

Processed prompts: 100%|██████████| 2/2 [00:00<00:00,  4.16it/s, est. speed input: 29.16 toks/s, output: 166.64 toks/s]


Generated Outputs:
------
Prompt:    'Hello, my name is'
Output:    ' Helen and I am from Shanghai. I am excited to introduce you to the city that I call home. Shanghai is a vibrant and dynamic city, known for its rich history, stunning architecture, and world'
------
Prompt:    'The president of the United States is'
Output:    ' elected through a complex process, with a multi-step process that involves several branches of the government. The process has evolved over time, and there have been controversies and disputes over the years. Here is an'
------
